#### Pytorch基础概念

科学计算工具包，一般两个核心问题：
1.怎么定义数据？torch.Tensor
2.怎么对数据操作？torch.autograd.Function

## 如何用PyTorch完成实验

常见的代码实现流程：
加载、预处理数据集、构建模型、定义损失函数、实现优化算法、迭代训练、加速计算（GPU）、存储模型、构建baseline

**加载、预处理数据集**

一旦选定了数据集，就要写一些函数去load数据集，然后presprocess数据集，normalize数据集。（可能是实验中占比最多的地方）
因为：每个数据集的格式都不太一样；需要告诉feed data；预处理和正则化的方式多种多样；

#### 常用数据集

vosion和text分别对应于视觉和自然语言处理数据集

In [4]:
import torchvision
minist = torchvision.datasets.MNIST(root = "./", download = True)

100%|███████████████████████████████████████████████████████████████████▉| 9912320/9912422 [00:27<00:00, 205356.90it/s]

Extracting ./MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST\raw



0it [00:00, ?it/s]
 57%|█████████████████████████████████████████▍                               | 16384/28881 [00:01<00:00, 32428.09it/s]
32768it [00:01, 30163.75it/s]                                                                                          

Extracting ./MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST\raw



0it [00:00, ?it/s]
 50%|██████████████████████████████████▎                                  | 819200/1648877 [00:10<00:07, 115819.98it/s]
9920512it [00:39, 205356.90it/s]                                                                                       
 76%|███████████████████████████████████████████████████▎                | 1245184/1648877 [00:13<00:02, 166939.29it/s]


 99%|███████████████████████████████████████████████████████████████████▌| 1638400/1648877 [00:14<00:00, 291413.04it/s]

Extracting ./MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST\raw




0it [00:00, ?it/s]

  0%|                                                                                         | 0/4542 [00:00<?, ?it/s]

8192it [00:00, 15883.37it/s]                                                                                           

Extracting ./MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST\raw
Processing...
Done!



1654784it [00:30, 291413.04it/s]                                                                                       

In [23]:
torchvision.datasets.mnist.MNIST

torchvision.datasets.mnist.MNIST

In [8]:
!pip install torchtext

In [27]:
import torchtext

WORD = torchtext.data.Field(init_token="<bos>",eos_token="<eos>")
UD_TAG = torchtext.data.Field(init_token="<bos>",eos_token="<eos>")
train, val, test = torchtext.datasets.UDPOS.splits(fields=(('word', WORD), ('udtag', UD_TAG), (None, None)))

downloading en-ud-v2.zip




.data\udpos\en-ud-v2.zip:   0%|                                                             | 0.00/688k [00:00<?, ?B/s]

.data\udpos\en-ud-v2.zip:   2%|█▏                                                  | 16.4k/688k [00:00<00:07, 91.2kB/s]

.data\udpos\en-ud-v2.zip:  10%|█████                                                | 65.5k/688k [00:00<00:05, 106kB/s]

.data\udpos\en-ud-v2.zip:  19%|██████████▎                                           | 131k/688k [00:00<00:04, 131kB/s]

.data\udpos\en-ud-v2.zip:  36%|███████████████████▎                                  | 246k/688k [00:00<00:02, 168kB/s]

.data\udpos\en-ud-v2.zip:  74%|███████████████████████████████████████▊              | 508k/688k [00:01<00:00, 224kB/s]

.data\udpos\en-ud-v2.zip:  81%|███████████████████████████████████████████▋          | 557k/688k [00:01<00:00, 210kB/s]

.data\udpos\en-ud-v2.zip: 100%|██████████████████████████████████████████████████████| 688k/688k [00:01<00:00, 460kB/s]

extracting


In [32]:
print(train.examples[0].word)
print(train.examples[0].udtag)

['Al', '-', 'Zaman', ':', 'American', 'forces', 'killed', 'Shaikh', 'Abdullah', 'al', '-', 'Ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'Qaim', ',', 'near', 'the', 'Syrian', 'border', '.']
['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT']


#### 快速构建自定义数据集
用`torch.utils.data`快速构建自定义数据集

In [36]:
import torch
import torch.utils.data as Data
x = torch.tensor([1, 2, 3])
y = torch.tensor([0, 0, 1])

dataset = Data.TensorDataset(x, y)
dataset.tensors

(tensor([1, 2, 3]), tensor([0, 0, 1]))

In [37]:
for x, y in dataset:
    print(x, y)

tensor(1) tensor(0)
tensor(2) tensor(0)
tensor(3) tensor(1)


#### 细粒度构建自定义数据集
继承自torch.utils.data.Dataset,然后重载函数len和getitem来构建

In [38]:
person_1 = ['高','富','帅','有']
person_2 = ['不高','富','帅','有']
person_3 = ['不高','不富','帅','有']
person_4 = ['不高','不富','不帅','无']

person = [person_1, person_2, person_3, person_4]

person

[['高', '富', '帅', '有'],
 ['不高', '富', '帅', '有'],
 ['不高', '不富', '帅', '有'],
 ['不高', '不富', '不帅', '无']]

In [40]:
import torch.utils.data as Data
class SingleDog(Data.Dataset):
    def __init__(self, person):
        # 读取文件之类的
        self.data = person
        
    def __getitem__(self, index):
        # 获取一个数据；再次可以写预处理；返回一个数据对（x,y）
        item = self.data[index]
        x = item[:3]    # 前三个
        y = item[-1]    # 最后一个
        return x, y
    
    def __len__(self):
        return len(self.data)

In [41]:
single = SingleDog(person)

In [42]:
for index, (x, y) in enumerate(single):
    print(f"迭代次数:{index}, x:{x}, y:{y}")

迭代次数:0, x:['高', '富', '帅'], y:有
迭代次数:1, x:['不高', '富', '帅'], y:有
迭代次数:2, x:['不高', '不富', '帅'], y:有
迭代次数:3, x:['不高', '不富', '不帅'], y:无


#### 加载数据集
使用torch.utils.data.DataLoader来加载数据集

为何要加载？加载后的数据集可以做batch,shuffle,还可以传入一个sample对象告诉是如何抽样的，等等。

In [45]:
tensor_person_data_loader = Data.DataLoader(dataset = person_dataset # 要加载哪个数据集
                                           batch_size = batch_size   # 一批数据有几个样例
                                           shuffle = True            # 需不需要随机洗牌
                                           sampler = sampler         # 准备怎么抽样
                                           collate_fn = my_collate_fn # 抽出来的样本，准备怎么处理)

SyntaxError: invalid syntax (<ipython-input-45-ff08d242c326>, line 2)

In [43]:
single_loader = Data.DataLoader(dataset = single, batch_size = 2)

In [44]:
for index, data in enumerate(single_loader):
    x, y = data
    print(f"迭代次数:{index}, x:{x}, y:{y}")

迭代次数:0, x:[('高', '不高'), ('富', '富'), ('帅', '帅')], y:('有', '有')
迭代次数:1, x:[('不高', '不高'), ('不富', '不富'), ('帅', '不帅')], y:('有', '无')


这个时候仍不能放到算法里运行，要把汉字变成我们要的tensor，可以使用collate

先定义一个字典：

In [46]:
word2id = {
    '高':1,
    '富':1,
    '帅':1,
    '不高':0,
    '不富':0,
    '不帅':0,
    '有':1,
    '无':0
}

In [71]:
# 新建一个数据集（通过字典将汉字转换成数字）
class TensorSingleDog(Data.Dataset):
    def __init__(self, person):
        # 读取文件之类的
        self.data = person
        
    def __getitem__(self, index):
        item = self.data[index]
        new_item = []
        for feature in item:
            new_item.append(word2id[feature])
        x = new_item[:3]
        y = new_item[-1]
        return x, y
    
    def __len__(self):
        return len(self.data)

In [72]:
single = TensorSingleDog(person)
single_loader = Data.DataLoader(dataset = single, batch_size = 2)

for index, data in enumerate(single_loader):
    x, y  = data
    print(f"迭代次数:{index},x:{x},y:{y}")

迭代次数:0,x:[tensor([1, 0]), tensor([1, 1]), tensor([1, 1])],y:tensor([1, 1])
迭代次数:1,x:[tensor([0, 0]), tensor([0, 0]), tensor([1, 0])],y:tensor([1, 0])


这时候变成了数字，然而我们传进来的具体的数据应该是一个tensor而非list

这时候就使用到`collate`，变为如下：（一个3*2的tensor）

In [73]:
def my_collate_fn(batch_data):
    x_batch = []
    y_batch = []
    for example in batch_data:
        x, y = example
        x_batch.append(x)
        y_batch.append(y)
    x_batch = torch.tensor(x_batch, dtype = torch.float32)
    y_batch = torch.tensor(y_batch, dtype = torch.float32)
    return x_batch, y_batch

In [75]:
single_loader = Data.DataLoader(dataset = single, batch_size = 2, collate_fn = my_collate_fn)

In [76]:
for index, data in enumerate(single_loader):
    x, y  = data
    print(f"迭代次数:{index},x:{x},y:{y}")

迭代次数:0,x:tensor([[1., 1., 1.],
        [0., 1., 1.]]),y:tensor([1., 1.])
迭代次数:1,x:tensor([[0., 0., 1.],
        [0., 0., 0.]]),y:tensor([1., 0.])


至此，数据预处理阶段完毕，这才是我们要输入到模型里面的x，y

## 二、构建模型
torchvision下面有常用的计算机视觉用到的模型，而没有nlp的模型（nlp没有统一的模型）

使用torch.hub.load()，来加载别人的模型，然而需要写github的人在根目录写一个配置文件。

#### 快速构建自己的神经网络结构

In [51]:
my_net = torch.nn.Sequential(
    torch.nn.Linear(1, 10),
    torch.nn.ReLU(inplace=True),
    torch.nn.Linear(10, 1)
)
my_net

Sequential(
  (0): Linear(in_features=1, out_features=10, bias=True)
  (1): ReLU(inplace=True)
  (2): Linear(in_features=10, out_features=1, bias=True)
)

In [52]:
my_net((torch.randn(10, 1)))

tensor([[0.1017],
        [0.6189],
        [0.4371],
        [0.1458],
        [0.0999],
        [1.1759],
        [0.1633],
        [0.1000],
        [0.1118],
        [0.1082]], grad_fn=<AddmmBackward>)

#### 快速构建多层

In [53]:
hun_layer = [torch.nn.Linear(10, 10) for _ in range(100)]
hun_layer

[Linear(in_features=10, out_features=10, bias=True),
 Linear(in_features=10, out_features=10, bias=True),
 Linear(in_features=10, out_features=10, bias=True),
 Linear(in_features=10, out_features=10, bias=True),
 Linear(in_features=10, out_features=10, bias=True),
 Linear(in_features=10, out_features=10, bias=True),
 Linear(in_features=10, out_features=10, bias=True),
 Linear(in_features=10, out_features=10, bias=True),
 Linear(in_features=10, out_features=10, bias=True),
 Linear(in_features=10, out_features=10, bias=True),
 Linear(in_features=10, out_features=10, bias=True),
 Linear(in_features=10, out_features=10, bias=True),
 Linear(in_features=10, out_features=10, bias=True),
 Linear(in_features=10, out_features=10, bias=True),
 Linear(in_features=10, out_features=10, bias=True),
 Linear(in_features=10, out_features=10, bias=True),
 Linear(in_features=10, out_features=10, bias=True),
 Linear(in_features=10, out_features=10, bias=True),
 Linear(in_features=10, out_features=10, bias=

以上我们用的都是现成的，没有做很好的处理，要想这样，看下面：

#### 深度自定义
想深度自定义（你的模型和别的模型不一样），可以去继承nn.Module

In [59]:
import torch.nn as nn

class SingleDogCls(nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(SingleDogCls, self).__init__()
        
        self.hidden = nn.Linear(n_feature, n_hidden)
        self.relu = nn.ReLU()
        self.predict = nn.Linear(n_hidden, n_output)
    
    def forward(self, x):
        x = self.hidden(x)
        x = self.relu(x)
        x = self.predict(x)
        return x

In [56]:
person

[['高', '富', '帅', '有'],
 ['不高', '富', '帅', '有'],
 ['不高', '不富', '帅', '有'],
 ['不高', '不富', '不帅', '无']]

In [63]:
will_u_be_single = SingleDogCls(n_feature = 3, n_hidden = 10, n_output = 1)

In [64]:
will_u_be_single

SingleDogCls(
  (hidden): Linear(in_features=3, out_features=10, bias=True)
  (relu): ReLU()
  (predict): Linear(in_features=10, out_features=1, bias=True)
)

In [66]:
# 画出神经网络结构
import hiddenlayer as hl
hl.build_graph(will_u_be_single, torch.randn([10, 3]))

ModuleNotFoundError: No module named 'hiddenlayer'

In [77]:
# 迭代去计算
for index, data in enumerate(single_loader):
    print(f"迭代次数：{index}")
    print(f"data:{data}")
    input_x, ground_truth = data
    predict = will_u_be_single(input_x)
    print(predict, ground_truth)

迭代次数：0
data:(tensor([[1., 1., 1.],
        [0., 1., 1.]]), tensor([1., 1.]))
tensor([[-0.5101],
        [-0.3515]], grad_fn=<AddmmBackward>) tensor([1., 1.])
迭代次数：1
data:(tensor([[0., 0., 1.],
        [0., 0., 0.]]), tensor([1., 0.]))
tensor([[-0.5553],
        [-0.4379]], grad_fn=<AddmmBackward>) tensor([1., 0.])


## 三、定义损失
度量，模型算出来的predict，和你的ground_truth之间的差距

一般用损失函数来度量，损失函数及其重要，如果度量的不准，那么整个模型学习的方向就会有偏差。官方定义的损失函数在目前有两个地方：
- torch.nn
- torch.nn.functional

torch.nn下面的loss函数，是作为模型的一部分存在的，实际上是torch.nn.funcitonal下loss函数的封装，实际上还是调用了后者下的函数

In [78]:
import torch.nn.functional as F

In [79]:
a = torch.tensor(2.)
b = torch.tensor(5.)

F.l1_loss(a, b) # |a - b|

tensor(3.)

In [80]:
F.mse_loss(a, b) # 均方误差

tensor(9.)

简单的两个数列之间的距离，可以用l1或者mse搞定，如果是两个分布，怎么度量呢？两个图像？两个类别？

torch.nn.functional下面，有这么多的损失函数：足够日常使用了

但是，如果你需要的loss，官方没有，你有两个方法实现自己的loss funtion
- 继承torch.nn.module实现loss
- 继承torch.autograd.funtion 实现loss，区别在于，你想把这个loss当作什么（模型还是函数）

In [ ]:
继承

## 四、实现优化算法
Pytorch集成了常见的优化算法，包括：
- torch.optim.SGD...

In [82]:
from torch.optim import Optimizer

optimizer = torch.optim.SGD(will_u_be_single.parameters(), lr = 0.05, momentum = 0.9)

如果你需要自定义优化算法，也可以继承torch.optim.Optimizer，通过实现其中的部分方法如step等，来优化模型的参数

In [ ]:
class MyOptimizer(Optimizer):
    def step()

调整学习率